<a href="https://colab.research.google.com/github/JazmineOrtizMarin/Simulaci-n-2/blob/main/Var_Control_Crudo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Variables de Control**

Aplicamos el método de **variables de control** para reducir la varianza en una estimación Monte Carlo de la integral:

$$ I = \int_0^1 \frac{1}{1+x^2} \, dx. $$

El valor real de esta integral es:

$$ I = \frac{\pi}{4} \approx 0.785398. $$

In [ ]:
# Librerías
import numpy as np
import random as rd

In [ ]:
# Función
def g(x):
  return 1 / (1 + x**2)

In [ ]:
n = 1000
r = 50

En el método Monte Carlo Crudo, generamos números aleatorios $$ U_i \sim U(0,1) $$ y evaluamos:

$$ \hat{\theta}_1 = \frac{1}{n} \sum_{i=1}^n g(U_i),
\quad \text{donde} \quad g(x) = \frac{1}{1+x^2}. $$

Este estimador es insesgado, pero su varianza puede ser alta.  
Para reducirla, se introduce una variable de control **Y** que cumpla:

- Está correlacionada con $$X = g(U)$$
- Tiene una esperanza conocida $$E[Y] = \mu_Y$$

El nuevo estimador se define como:

$$Z = X + c\,(Y - \mu_Y),$$

donde \(c\) es una constante que se elige para minimizar la varianza de **Z**.

In [ ]:
# Método crudo
def crudo(n):
    G = []
    for i in range(n):
        u = rd.random()
        G.append(g(u))
    return np.mean(G), np.var(G), np.std(G), G

En este ejercicio, elegimos como variable de control el promedio de los mismos números uniformes:

$$Y = \frac{1}{n} \sum_{i=1}^n U_i$$

porque sabemos que si $$U_i \sim U(0,1)$$ entonces

$$E[Y] = \mu_Y = \frac{1}{2}$$

La varianza de **Z** se puede expresar como:

$$\text{Var}(Z) = \text{Var}(X) + 2c\,\text{Cov}(X,Y) + c^2\,\text{Var}(Y)$$

Minimizando respecto a **c**:

$$c^* = -\frac{\text{Cov}(X,Y)}{\text{Var}(Y)}$$

Sustituyendo este valor se obtiene la varianza mínima alcanzable:

$$\text{Var}(Z) = \text{Var}(X) -
\frac{\text{Cov}(X,Y)^2}{\text{Var}(Y)}$$

Por lo tanto, **la varianza disminuye** siempre que exista correlación
entre **X** y **Y**.

In [ ]:
def control_variate(n, r):

    estimaciones_X = []
    estimaciones_Y = []
    estimaciones_Z = []

    for _ in range(r):
        U = [rd.random() for _ in range(n)] # uniformes
        X = np.mean([g(u) for u in U]) # estimador base
        Y = np.mean(U) # variable de control
        estimaciones_X.append(X)
        estimaciones_Y.append(Y)

    # Calculo de medias
    mu_X = np.mean(estimaciones_X)
    mu_Y = np.mean(estimaciones_Y)
    var_Y = np.var(estimaciones_Y)
    cov_XY = np.cov(estimaciones_X, estimaciones_Y, bias=True)[0,1]

    # constante óptima
    c = - cov_XY / var_Y

    # estimaciones ajustadas
    for i in range(r):
        Z = estimaciones_X[i] + c * (estimaciones_Y[i] - 0.5)
        estimaciones_Z.append(Z)

    return {
        "c": c,
        "media_X": np.mean(estimaciones_X),
        "var_X": np.var(estimaciones_X),
        "media_Z": np.mean(estimaciones_Z),
        "var_Z": np.var(estimaciones_Z)
    }

Se calculan las estimaciones crudas $$\hat{X} \text{ y } \hat{Y}$$ en 50 repeticiones, se obtienen la covarianza y la varianza, y con ello se calcula el valor óptimo de **c**.

Cada nueva estimación se corrige aplicando:

$$\hat{Z}_i = \hat{X}_i + c^*(\hat{Y}_i - 0.5)$$

El promedio de las $$\hat{Z}_i$$ será nuestro estimador con control de varianza.

In [ ]:
res = control_variate(n, r)

In [ ]:
print("Método de Variables de Control")
print("-----------------------------------------------")
print(f"Constante óptima c:          {res['c']:.10f}")
print(f"Media (crudo):               {res['media_X']:.10f}")
print(f"Varianza (crudo):            {res['var_X']:.10f}")
print(f"Media (con control):         {res['media_Z']:.10f}")
print(f"Varianza (con control):      {res['var_Z']:.10f}")
print("-----------------------------------------------")
print("Valor real de la integral:   π/4 =", np.pi/4)

Método de Variables de Control
-----------------------------------------------
Constante óptima c:          0.5487514896
Media (crudo):               0.7852933526
Varianza (crudo):            0.0000343449
Media (con control):         0.7855400439
Varianza (con control):      0.0000003048
-----------------------------------------------
Valor real de la integral:   π/4 = 0.7853981633974483
